In [ ]:
#importing packages

import schedule
import time
import smtplib
import ssl
import pandas as pd
from datetime import datetime
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
import matplotlib.pyplot as plt
from email.message import EmailMessage
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders

data=pd.read_excel('Grades.xlsx')
num_rows=data.shape[0]
num_columns=data.shape[1]

#mathematical operation

data['Total_Score'] = data.sum(axis=1)
output_file = 'python_excel_automation.xlsx'
with pd.ExcelWriter(output_file) as writer:
    # Save the DataFrame as a sheet named 'Sheet1' in the workbook
    data.to_excel(writer, sheet_name='Arithmetics', index=False)

print("DataFrame saved to Excel file as sheet 'Arithmetics':", output_file)

# If-else conditioning
num_rows = data.shape[0]
for i in range(0, num_rows):  # Corrected the range
    if data['English'].iloc[i] > data['Hindi'].iloc[i]:  # Corrected indexing and comparison
        data.at[i, 'Result'] = "The student is good in English"  # Corrected assignment
        
    else:
        data.at[i, 'Result'] = "The student needs improvement"  # Corrected assignment
        
        
data['Rank']=data['Total_Score'].rank(method='max',ascending=False)

def highlight_result(data):
    # Assuming the column 'Result' is present in the DataFrame
    # Also, fixing the typo in the condition ("students" -> "student")
    if data['Result'] == "The student is good in English":
        return ['background-color: Green'] * len(data)
    else:
        return ['background-color: Yellow'] * len(data)

# Create a copy of the DataFrame and apply styling to the copy
styled_data = data.copy()
styled_data = styled_data.style.apply(highlight_result, axis=1)

output_file = 'python_excel_automation.xlsx'
with pd.ExcelWriter(output_file, engine='openpyxl', mode='a') as writer:
    # Save the DataFrame as a new sheet in the existing workbook
    styled_data.to_excel(writer, sheet_name='Conditionals', index=False)

print("DataFrame saved to Excel file as sheet 'Conditionals' in existing workbook:", output_file)

print("******************************************************")



today_date = datetime.today().date()



subject=f"Analytics Report: {today_date}"
body = "Hi All! Please find attached the report."
sender_email = 'Sender Email'
sender_password = 'Sender Email Password'
receiver_email = ['Receiver Email']


message=EmailMessage()
message["From"]=sender_email
message["To"]=receiver_email
message["Subject"]=subject
end="Thanks!"

html=f"""

<html>
    <body>
        <h1>{body}</h1>
        <p>{end}</p>
    </body>
</html>
"""


message.add_alternative(html,subtype="html")

exl_filename = "python_excel_automation.xlsx"
with open(exl_filename, "rb") as f:
    part = MIMEBase("application", "octet-stream")
    part.set_payload(f.read())
    encoders.encode_base64(part)
    part.add_header("Content-Disposition", f"attachment; filename={exl_filename}")
    message.attach(part)
    

context=ssl.create_default_context()
print("Code block executed successfully!!")
def send_emails():
    context=ssl.create_default_context()
    with smtplib.SMTP_SSL("smtp.gmail.com",465,context=context) as server:
        server.login(sender_email,sender_password)
        server.sendmail(sender_email,receiver_email,message.as_string())

    

schedule.every(5).minutes.do(send_emails)
while 1:
    schedule.run_pending()
    time.sleep(60)      
 